In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [2]:
# load data
tree_data = pd.read_csv('data/tree.csv')
genes_data = pd.read_csv('data/vert_genes.csv')

print(tree_data)
print(genes_data)


     Parent  Child     age_ch          t                 species
0     222.0      1   0.000000   9.000250  Peromyscus_maniculatus
1     222.0      2   0.000000   9.000250            Mus_musculus
2     221.0      3   0.000000  12.172706      Cricetulus_griseus
3     220.0      4   0.000000  14.684269       Rattus_norvegicus
4     219.0      5   0.000000  17.062881    Mesocricetus_auratus
..      ...    ...        ...        ...                     ...
402   399.0    403   7.861905   1.640772                     NaN
403   403.0    404   5.750020   2.111885                     NaN
404   404.0    405   5.500000   0.250020                     NaN
405   403.0    406   5.000000   2.861905                     NaN
406     NaN    407  50.024697   2.091292                     NaN

[407 rows x 5 columns]
        ensembl_id orthId  glength                       species
0     XP_009078534  1CPN2    66213          Acanthisitta_chloris
1     XP_005013863  1CPN2    44913            Anas_platyrhynchos
2

In [3]:
class Node:
    def __init__(self,data):
        self.l = None
        self.r = None
        self.data = data


In [4]:
def create_tree(root):
    children = tree_data[tree_data['Parent']==root]
    node = Node(tree_data[tree_data['Child']==root])
    if len(children)>0:
        for i in range(0, len(children)):
            if i == 0:
                node.l = create_tree(children.iloc[1].iloc[1])
            else:
                node.r = create_tree(children.iloc[0].iloc[1])
        return node
    else:
        return node



In [5]:
# gamma_0 + (alpha * time) (beta * parent gene len) + (gamma * time * parent gene len)

# Parameters
# for Z_0
alpha_0 = 50000
variance_0 = 5000

# for Z_i and X_i
alpha = 0
beta = 1.01
variance = 2500 



In [6]:
tree = create_tree(407)

mean = np.zeros(407)
covariance = np.array(407*[np.zeros(407)])

mean[406] = alpha_0
covariance[406,406] = variance_0

def meanCovariance(tree, mean, covariance):
    if tree != None:
        index = int(tree.data.iloc[0].iloc[1])
        t = int(tree.data.iloc[0].iloc[3])
        parent = int(tree.data.iloc[0].iloc[0])
        mean[index-1] = mean[parent-1]*beta+alpha*t

        covariance[index-1,:] = covariance[parent-1,:]*beta 
        covariance[:,index-1] = covariance[parent-1,:]*beta 

        covariance[index-1,index-1] = variance*t+beta*covariance[parent-1,parent-1]
        '''covariance[index-1,parent-1] = beta**2*covariance[parent-1,parent-1]
        covariance[parent-1,index-1] = beta**2*covariance[parent-1,parent-1]'''
        meanCovariance(tree.l,mean,covariance)
        meanCovariance(tree.r,mean,covariance)
    else:
        return

meanCovariance(tree.l,mean,covariance)
meanCovariance(tree.r,mean,covariance)

print(covariance)
print(mean)


[[121595.59978422 100086.55578206  92435.80578206 ...   6412.15997509
    6348.67324266   5980.73737843]
 [100086.55578206 121595.59978422  92435.80578206 ...   6412.15997509
    6348.67324266   5980.73737843]
 [ 92435.80578206  92435.80578206 120614.4552319  ...   6348.67324266
    6285.81509174   5921.52215686]
 ...
 [  6412.15997509   6412.15997509   6348.67324266 ... 115331.12154579
  111383.93276125   5360.67676054]
 [  6348.67324266   6348.67324266   6285.81509174 ... 111383.93276125
  114189.22925326   5307.60075301]
 [  5980.73737843   5980.73737843   5921.52215686 ...   5360.67676054
    5307.60075301   5000.        ]]
[59807.37378433 59807.37378433 59215.22156865 58628.93224618
 58048.4477685  57473.71066188 56904.66402166 56341.2515066
 55783.41733327 56341.2515066  56341.2515066  56904.66402166
 56904.66402166 56341.2515066  54684.26363422 62858.15091742
 62858.15091742 62235.79298755 61619.59701737 61009.5019974
 61009.5019974  61009.5019974  60405.44752218 60405.44752218


In [7]:
#fik lige en bug her så det er kommenteret ud, men bruger det heller ikke lige pt. skal dog bruge det til message passing
J = np.linalg.inv(covariance)
h = np.matmul(J,mean)
print(J)
print(h)

[[ 4.64920710e-05 -2.33739621e-19  7.01920730e-20 ... -1.78067566e-19
  -6.65134968e-20  2.92464515e-19]
 [-2.01225603e-19  4.64920710e-05  8.77683511e-20 ... -2.07435410e-19
  -6.64371181e-20  2.95100237e-19]
 [ 5.10325830e-20  7.36815600e-20  3.43715188e-05 ...  7.04905756e-20
   3.08406824e-20 -1.28709976e-19]
 ...
 [-2.97261783e-19 -2.84134784e-19  1.17358896e-20 ... -6.87079945e-04
   3.27494250e-18 -2.35394319e-19]
 [-8.85549506e-20 -8.34333982e-20  4.49039727e-20 ...  2.49968856e-18
   4.35684071e-04  2.65622469e-19]
 [ 1.99305198e-20  2.06656620e-20 -1.11659921e-19 ... -7.70817864e-19
  -4.19981656e-19  7.53387702e-04]]
[ 5.98822038e-15 -7.64869552e-15  5.34129084e-14 -5.90906570e-15
 -1.73680612e-14  1.19504466e-14 -3.45948896e-14 -1.68818138e-14
  1.39268317e-14 -1.17416715e-14 -5.51267033e-15 -1.61917615e-14
  1.67961290e-14 -2.11021408e-15  4.48666562e-15  1.35625577e-15
  2.96984856e-14  2.31407971e-14 -2.16064628e-14 -2.86372715e-14
  2.05963217e-15  4.43381300e-15  2.883

In [8]:
#load gene data og få lagt det ind i arrays, hvis et gene ikke er der sættes dens længde til 0.
a = np.array(10*[np.zeros(204)])
unique_genes = genes_data.loc[:,'orthId'].unique()

for i in range(0,204):
    species = tree_data.iloc[i].iloc[4]
    extract = genes_data[genes_data['species']==species]
    for j in range(0,10):
        temp = extract[extract['orthId']==unique_genes[j]]
        if len(temp)==1:
            a[j,i] = temp['glength']

print(a)
print(a.shape)

[[ 88338. 219992.  89972. ...   4568.   4239.  66937.]
 [ 30935.  31081.  40971. ...  34954.  33799.  31589.]
 [ 21555.  20662.  12707. ...   8561.   8475.      0.]
 ...
 [ 19977.  21211.  24187. ...   3964.      0.      0.]
 [ 16494.  15176.  16651. ...   7387.  19106.  18206.]
 [ 29924.   8341.   6260. ...   1881.   1852.   1778.]]
(10, 204)


In [9]:
#estimere P(Z_i|x_1,..,x_204) og print normalfordelingen med (mean,variance)
for j in range(406,407):
    sigmaYX = covariance[j,:204]
    sigmaXY = covariance[:204,j]
    sigmaXX = np.linalg.inv(covariance[:204,:204])
    meanY = mean[j]+np.matmul(np.matmul(sigmaYX,sigmaXX),a[0]-mean[:204])
    covarianceY = covariance[j,j]-np.matmul(np.matmul(sigmaYX,sigmaXX),sigmaXY)
    print((meanY,covarianceY))


(57982.801007587106, 3357.478507499791)


In [10]:

def conditionLeaf(tree,J,h,a):
    if tree!=None:
        leafL = False
        leafR = False
        index = int(tree.data.iloc[0].iloc[1])-1
        if tree.l.l == None: #all nodes have two children
            leafL = True
            indexChildL = int(tree.l.data.iloc[0].iloc[1])-1
            J[index,index] = J[index,index]
            h[index] = h[index]-J[index,indexChildL]*a[indexChildL]
        if tree.r.l == None: #all nodes have two children
            leafR = True
            indexChildR = int(tree.r.data.iloc[0].iloc[1])-1
            J[index,index] = J[index,index]
            h[index] = h[index]-J[index,indexChildR]*a[indexChildR]
        if leafL == False:
            conditionLeaf(tree.l,J,h,a)
        if leafR == False:   
            conditionLeaf(tree.r,J,h,a)
    else:
        return


'''conditionLeaf(tree,J,h,a[0])
for i in range(0,204):
    h[i]=0
    J[i,:] = np.zeros(407)
    J[:,i] = np.zeros(407)
print(J[221,221]**(-1))
print(J[221,221]**(-1)*h[221])'''

'conditionLeaf(tree,J,h,a[0])\nfor i in range(0,204):\n    h[i]=0\n    J[i,:] = np.zeros(407)\n    J[:,i] = np.zeros(407)\nprint(J[221,221]**(-1))\nprint(J[221,221]**(-1)*h[221])'

In [11]:
'''def removeLeaf(tree):
    if tree.l.l == None:
        tree.l = None
    elif tree != None and tree.l != None:
        removeLeaf(tree.l)
    if tree.r.l == None:
        tree.r = None
    elif tree != None and tree.l != None: 
        removeLeaf(tree.r)

treeNoLeaf = copy.deepcopy(tree)
print(treeNoLeaf.data)
removeLeaf(treeNoLeaf)'''



'def removeLeaf(tree):\n    if tree.l.l == None:\n        tree.l = None\n    elif tree != None and tree.l != None:\n        removeLeaf(tree.l)\n    if tree.r.l == None:\n        tree.r = None\n    elif tree != None and tree.l != None: \n        removeLeaf(tree.r)\n\ntreeNoLeaf = copy.deepcopy(tree)\nprint(treeNoLeaf.data)\nremoveLeaf(treeNoLeaf)'

In [12]:

Jhat = np.zeros(407)
Hhat = np.zeros(407)
messageArray = np.zeros(407)
Jmessage = np.zeros(407)
Hmessage = np.zeros(407)

def message(tree, i,Jhat,Hhat, J, h, Jmessage, Hmessage):
    if tree != None:
        c = cluster(tree,Jhat,Hhat, J, h, Jmessage, Hmessage)
        Jmessage[i] += -1*J[j,i]*c[0]**(-1)*J[i,j]
        Hmessage[i] += -1*J[j,i]*c[0]**(-1)*c[1]
        return (Jmessage[i],Hmessage[i])
    else:
        return (0,0)
    
def cluster(tree,Jhat,Hhat, J, h, Jmessage, Hmessage):
    if tree != None:
        i = int(tree.data.iloc[0].iloc[1])-1
        if tree.l.l != None:
            message(tree.l,i,Jhat,Hhat, J, h, Jmessage, Hmessage)
        if tree.r.r != None:
            message(tree.r,i,Jhat,Hhat, J, h, Jmessage, Hmessage)
        Jhat[i] = J[i,i] + Jmessage[i]
        Hhat[i] = h[i] + Hmessage[i]
        return(Jhat[i],Hhat[i])
    
    
#cluster(tree,Jhat,Hhat,J,h, Jmessage, Hmessage)
print(Jhat)
#print(Hhat)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [19]:
JhatD = np.zeros(407) #by D, down, its meant from the root towards leaf
HhatD = np.zeros(407)
JmessageD = np.zeros(407)
HmessageD = np.zeros(407)

def messageD(tree, i,JhatD,HhatD, J, h, Jmessage, Hmessage):
    if tree != None:
        j = int(tree.data.iloc[0].iloc[1])-1
        print(J[j,i])
        Jmessage[j] += -1*J[j,i]*JhatD[i]**(-1)*J[i,j]
        Hmessage[j] += -1*J[j,i]*JhatD[i]**(-1)*HhatD[i]
    
def clusterD(tree,JhatD,HhatD, J, h, JmessageD, HmessageD):
    if tree.l != None:
        i = int(tree.data.iloc[0].iloc[1])-1
        iL = int(tree.l.data.iloc[0].iloc[1])-1
        iR = int(tree.r.data.iloc[0].iloc[1])-1
        
        
        JhatD[i] += J[i,i]
        HhatD[i] += h[i]
        messageD(tree.l, i,JhatD,HhatD, J, h, JmessageD, HmessageD)
        messageD(tree.r, i,JhatD,HhatD, J, h, JmessageD, HmessageD)

        JhatD[iL] += JmessageD[iL]
        JhatD[iR] += JmessageD[iR]
        HhatD[iL] += HmessageD[iL]
        HhatD[iR] += HmessageD[iR]
        clusterD(tree.l,JhatD,HhatD, J, h, JmessageD, HmessageD)
        clusterD(tree.r,JhatD,HhatD, J, h, JmessageD, HmessageD)
    
    
clusterD(tree,Jhat,Hhat,J,h, Jmessage, Hmessage)
print(JmessageD)

-0.00013557956909859722
-0.00041232904674423323
-0.0002072540986550032
-8.792299721314573e-06
-0.00020947935782630102
-9.410977411027094e-06
-2.904870476939901e-05
-0.0004450923060176467
-1.699953662806699e-05
-2.7361349368154863e-05
-0.00026568334100528966
-0.0002656833410052888
-4.691397113634136e-05
-3.807187730876817e-05
-0.0002663638615497732
-0.0002663638615497723
-5.8579134170451295e-05
-0.0002130067332785095
-5.9194831359215244e-05
-0.00010562311875108232
-2.7385245759306006e-05
-1.7008757836647542e-05
-5.316842984287367e-05
-5.316842984287368e-05
-1.454138693747949e-05
-1.4541386937479465e-05
-4.551897162171596e-05
-0.0001405012041935473
-8.446097096860989e-05
-1.5084040659407292e-05
-3.173454302051461e-05
-1.8591289114690666e-05
-0.00015558728502039978
-5.31815940462421e-05
-8.855222783870811e-05
-8.855222783870842e-05
-0.0002191038611985659
-1.193716757301765e-05
0.0022711064929049414
-0.00022172058960710659
-1.9404107288105682e-05
-0.0004924821667008723
-4.205513179563202e-

In [20]:
for i in range(220,225):
    varianceHat = Jhat[i]**(-1)
    meanHat = varianceHat*Hhat[i]
    print((meanHat, varianceHat))
    

(-362.1401739680238, 2118.5811964706218)
(-244.33750437262168, 3328.701251262044)
(-12.168735179711577, 577.9883379040127)
(-6.326035237687008, 1027.219692350436)
(2.9065104503300088, -290.01712392996546)
